In [1]:
import cv2
import numpy as np
import time
import mediapipe as mp
import matplotlib.pyplot as plt
import pyttsx3
engine = pyttsx3.init()
engine.setProperty('rate',200)
voice_id = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
engine.setProperty('voice', voice_id)

In [2]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for images and videos.
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)
hands_videos = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

In [3]:
def detectHandsLandmarks(image, hands, draw=True, display = True):
    '''
    This function performs hands landmarks detection on an image.
    Args:
        image:   The input image with prominent hand(s) whose landmarks needs to be detected.
        hands:   The Hands function required to perform the hands landmarks detection.
        draw:    A boolean value that is if set to true the function draws hands landmarks on the output image. 
        display: A boolean value that is if set to true the function displays the original input image, and the output 
                 image with hands landmarks drawn if it was specified and returns nothing.
    Returns:
        output_image: A copy of input image with the detected hands landmarks drawn if it was specified.
        results:      The output of the hands landmarks detection on the input image.
    '''
    
    # Create a copy of the input image to draw landmarks on.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Hands Landmarks Detection.
    results = hands.process(imgRGB)
    
    # Check if landmarks are found and are specified to be drawn.
    if results.multi_hand_landmarks and draw:
        
        # Iterate over the found hands.
        for hand_landmarks in results.multi_hand_landmarks:
            
            # Draw the hand landmarks on the copy of the input image.
            mp_drawing.draw_landmarks(image = output_image, landmark_list = hand_landmarks,
                                      connections = mp_hands.HAND_CONNECTIONS,
                                      landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                                   thickness=2, circle_radius=2))
    
    # Check if the original input image and the output image are specified to be displayed.
    if display:
        
        # Display the original input image and the output image.
        plt.figure(figsize=[15,15])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off');
        #image[height:width:depth:slicing of which image to display]->This is for 3D images gives 3D array
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output");plt.axis('off');
        
    # Otherwise
    else:
        
        # Return the output image and results of hands landmarks detection.
        return output_image, results

In [4]:
def countFingers(image, results, draw=True, display=True):
    '''
    This function will count the number of fingers up for each hand in the image.
    Args:
        image:   The image of the hands on which the fingers counting is required to be performed.
        results: The output of the hands landmarks detection performed on the image of the hands.
        draw:    A boolean value that is if set to true the function writes the total count of fingers of the hands on the
                 output image.
        display: A boolean value that is if set to true the function displays the resultant image and returns nothing.
    Returns:
        output_image:     A copy of the input image with the fingers count written, if it was specified.
        fingers_statuses: A dictionary containing the status (i.e., open or close) of each finger of both hands.
        count:            A dictionary containing the count of the fingers that are up, of both hands.
    '''
    
    # Get the height and width of the input image.
    height, width, _ = image.shape
    
    # Create a copy of the input image to write the count of fingers on.
    output_image = image.copy()
    
    # Initialize a dictionary to store the count of fingers of both hands.
    count = {'RIGHT': 0, 'LEFT': 0}
    
    # Store the indexes of the tips landmarks of each finger of a hand in a list.
    fingers_tips_ids = [mp_hands.HandLandmark.INDEX_FINGER_TIP, mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
                        mp_hands.HandLandmark.RING_FINGER_TIP, mp_hands.HandLandmark.PINKY_TIP]
    
    # Initialize a dictionary to store the status (i.e., True for open and False for close) of each finger of both hands.
    fingers_statuses = {'RIGHT_THUMB': False, 'RIGHT_INDEX': False, 'RIGHT_MIDDLE': False, 'RIGHT_RING': False,
                        'RIGHT_PINKY': False, 'LEFT_THUMB': False, 'LEFT_INDEX': False, 'LEFT_MIDDLE': False,
                        'LEFT_RING': False, 'LEFT_PINKY': False}
    
    
    # Iterate over the found hands in the image.
    for hand_index, hand_info in enumerate(results.multi_handedness):
        
        # Retrieve the label of the found hand.
        hand_label = hand_info.classification[0].label
        
        # Retrieve the landmarks of the found hand.
        hand_landmarks =  results.multi_hand_landmarks[hand_index]
        
        # Iterate over the indexes of the tips landmarks of each finger of the hand.
        for tip_index in fingers_tips_ids:
            
            # Retrieve the label (i.e., index, middle, etc.) of the finger on which we are iterating upon.
            finger_name = tip_index.name.split("_")[0]
            
            # Check if the finger is up by comparing the y-coordinates of the tip and pip landmarks.
            if (hand_landmarks.landmark[tip_index].y < hand_landmarks.landmark[tip_index - 2].y):
                
                # Update the status of the finger in the dictionary to true.
                fingers_statuses[hand_label.upper()+"_"+finger_name] = True
                
                # Increment the count of the fingers up of the hand by 1.
                count[hand_label.upper()] += 1
        
        # Retrieve the y-coordinates of the tip and mcp landmarks of the thumb of the hand.
        thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
        thumb_mcp_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP - 2].x
        
        # Check if the thumb is up by comparing the hand label and the x-coordinates of the retrieved landmarks.
        if (hand_label=='Right' and (thumb_tip_x < thumb_mcp_x)) or (hand_label=='Left' and (thumb_tip_x > thumb_mcp_x)):
            
            # Update the status of the thumb in the dictionary to true.
            fingers_statuses[hand_label.upper()+"_THUMB"] = True
            
            # Increment the count of the fingers up of the hand by 1.
            count[hand_label.upper()] += 1
     
    # Check if the total count of the fingers of both hands are specified to be written on the output image.
    if draw:

        # Write the total count of the fingers of both hands on the output image.
        #cv2.putText(output_image, " Total Fingers: ", (10, 25),cv2.FONT_HERSHEY_COMPLEX, 1, (20,255,155), 2)
        cv2.putText(output_image, str(sum(count.values())), (width//150,240), cv2.FONT_HERSHEY_SIMPLEX,
                    5, (20,255,155))

    # Check if the output image is specified to be displayed.
    if display:
        
        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    
    # Otherwise
    else:

        # Return the output image, the status of each finger and the count of the fingers up of both hands.
        return output_image, fingers_statuses, count

In [5]:
def recognizeGestures(image, fingers_statuses, results, count, draw=True, display=True):
    '''
    This function will determine the gesture of the left(index=0) and right(index=1) hand in the image.
    Args:
        image:            The image of the hands on which the hand gesture recognition is required to be performed.
        fingers_statuses: A dictionary containing the status (i.e., open or close) of each finger of both hands. 
        count:            A dictionary containing the count of the fingers that are up, of both hands.
        draw:             A boolean value that is if set to true the function writes the gestures of the hands on the
                          output image, after recognition.
        display:          A boolean value that is if set to true the function displays the resultant image and 
                          returns nothing.
    Returns:
        output_image:   A copy of the input image with the left and right hand recognized gestures written if it was 
                        specified.
        hands_gestures: A dictionary containing the recognized gestures of the right and left hand.
    '''
    
    # Create a copy of the input image.
    output_image = image.copy()
    h, w, c = output_image.shape
    finger_tips = [8, 12, 16, 20]
    thumb_tip = 4
    # Store the labels of both hands in a list.
    #label = 'results.multi_handedness[0].classification[0].label'
    hands_labels = ['RIGHT']
    
    # Initialize a dictionary to store the gestures of both hands in the image.
    hands_gestures = {'RIGHT': "UNKNOWN"}
    #hands_gestures1 = {'LEFT':'UNKNOWN'}
    
    
    # Iterate over the left and right hand.
    for hand_index, hand_label in enumerate(hands_labels):
        
        # Initialize a variable to store the color we will use to write the hands gestures on the image.
        # Initially it is red which represents that the gesture is not recognized.
        color = (0, 0, 255)
        
          #check if the person is making the 'UNITE' gesture with the hand.
        ####################################################################################################################
        lm_list2 = []
        # Check if the number of fingers up is 0.
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                
                for id, lm in enumerate(hand_landmark.landmark):
                    lm_list2.append(lm)
                finger_fold_status = []
                for tip in finger_tips:
                    if lm_list2[tip].y > lm_list2[tip - 3].y:
                        finger_fold_status.append(True)
                    else:
                        finger_fold_status.append(False)
                if all(finger_fold_status):
                    if lm_list2[thumb_tip].x > lm_list2[thumb_tip - 1].x > lm_list2[thumb_tip - 2].x:
                        hands_gestures[hand_label]="UNITE"
                        print("UNITE")
                        color=(0,255,0)
                        engine.say("UNITE")
                        engine.runAndWait()
                        engine.stop()
                        #cv2.putText(output_image,"UNITE" , (10, (hand_index+1) * 60),cv2.FONT_HERSHEY_PLAIN, 4, color, 5)
        
        # Check if the person is making the 'VICTORY' gesture with the hand.
        ####################################################################################################################
        
        # Check if the number of fingers up is 2 and the fingers that are up, are the index and the middle finger.
        if count[hand_label] == 2  and fingers_statuses[hand_label+'_MIDDLE'] and fingers_statuses[hand_label+'_INDEX']:
            
            # Update the gesture value of the hand that we are iterating upon to V SIGN.
            
            hands_gestures[hand_label] = "VICTORY"
            
            # Update the color value to green.
            color=(0,255,0)
            engine.say("Victory")
            engine.runAndWait()
            engine.stop()
            
        ####################################################################################################################            
        
         # Check if the person is making the 'CALL ME' gesture with the hand.
        ####################################################################################################################
        
        # Check if the number of fingers up is 2 and the fingers that are up, are the pinky and the thumb.
        if count[hand_label] == 2  and fingers_statuses[hand_label+'_THUMB'] and fingers_statuses[hand_label+'_PINKY']:
            
            # Update the gesture value of the hand that we are iterating upon to call me.
            hands_gestures[hand_label] = "CALL ME"
            
            # Update the color value to green.
            color=(0,255,0)
            engine.say("CALL ME")
            engine.runAndWait()
            engine.stop()
         ##########################################################################################################################################################
        
        
        # Check if the person is making the 'ROCK ON' gesture with the hand.
        ##########################################################################################################################################################
        
        # Check if the number of fingers up is 3 and the fingers that are up, are the thumb, index and the pinky finger.
        if count[hand_label] == 3 and fingers_statuses[hand_label+'_THUMB'] and fingers_statuses[hand_label+'_INDEX'] and fingers_statuses[hand_label+'_PINKY']:
                
            # Update the gesture value of the hand that we are iterating upon to SPIDERMAN SIGN.
            hands_gestures[hand_label] = "ROCK ON"

            # Update the color value to green.
            color=(0,255,0)
            engine.say("ROCK ON")
            engine.runAndWait()
            engine.stop()
                
        ##########################################################################################################################################################
        
        
         # Check if the person is making the 'PEACE' gesture with the hand.
        ##########################################################################################################################################################
        
        # Check if the number of fingers up is 3 and the fingers that are up, are the thumb, index and the pinky finger.
        if count[hand_label] == 3 and fingers_statuses[hand_label+'_INDEX'] and fingers_statuses[hand_label+'_MIDDLE'] and fingers_statuses[hand_label+'_RING']:
                
            # Update the gesture value of the hand that we are iterating upon to SPIDERMAN SIGN.
            hands_gestures[hand_label] = "PEACE"

            # Update the color value to green.
            color=(0,255,0)
            engine.say("PEACE")
            engine.runAndWait()
            engine.stop()
                
        ##########################################################################################################################################################
        
        
        # Check if the person is making the 'HELLO' gesture with the hand.
        ####################################################################################################################
        
        # Check if the number of fingers up is 5, which means that all the fingers are up.
        if count[hand_label] == 5:
            
            # Update the gesture value of the hand that we are iterating upon to HIGH-FIVE SIGN.
            hands_gestures[hand_label] = "HELLO"
            
            # Update the color value to green.
            color=(0,255,0)
            engine.say("HELLO")
            engine.runAndWait()
            engine.stop()
       
        ####################################################################################################################  

        # Check if the person is making the 'THUMBS UP' and 'THUMBS DOWN' gesture with the hand.
        ####################################################################################################################
        lm_list = []
        # Check if the number of fingers up is 1, which means THUMB is up
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:    
                for id, lm in enumerate(hand_landmark.landmark):
                    lm_list.append(lm)
                finger_fold_status = []
                for tip in finger_tips:
                    if lm_list[tip].x < lm_list[tip - 3].x:
                        finger_fold_status.append(True)
                    else:
                        finger_fold_status.append(False)
                print(finger_fold_status)
                if all(finger_fold_status):
                    # thumbs up
                    if lm_list[thumb_tip].y < lm_list[thumb_tip - 1].y < lm_list[thumb_tip - 2].y :
                        hands_gestures[hand_label]='THUMBS UP'
                        print("THUMBS UP")
                        color=(0,255,0)
                        engine.say("THUMBS UP")
                        engine.runAndWait()
                        engine.stop()
                        #cv2.putText(output_image,"THUMBS UP" , (10, (hand_index+1) * 60),
                          #          cv2.FONT_HERSHEY_PLAIN, 4, color, 5)

                    # Thumbs down
                    if lm_list[thumb_tip].y > lm_list[thumb_tip - 1].y > lm_list[thumb_tip - 2].y:
                        hands_gestures[hand_label]='THUMBS DOWN'
                        #cv2.putText(output_image, "THUMBS DOWN" , (10, (hand_index+1) * 60),
                         #           cv2.FONT_HERSHEY_PLAIN, 4, color, 5)
                        print("THUMBS DOWN")
                        color=(0,255,0)
                        engine.say("THUMBS DOWN")
                        engine.runAndWait()
                        engine.stop()
                    
                    
                     # Retrieve the y-coordinates of the tip and mcp landmarks of the thumb of the hand.
                    #thumb_tip_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x
                    #thumb_mcp_x = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP - 2].x

                    # Check if the thumb is up by comparing the hand label and the x-coordinates of the retrieved landmarks.
                    '''
                    if ((hands_gestures.items=='Left')):

                        if lm_list[tip].x < lm_list[tip - 3].x:
                            cv2.circle(output_image, (x, y), 15, (0, 255, 0), cv2.FILLED)
                            finger_fold_status.append(True)
                            if all(finger_fold_status):
                                # THHUMBS UP
                                if results.hand_landmarks.landmark[hand_landmark.thumb_tip]:
                                    print("THUMBS UP")
                                    cv2.putText(output_image, "THUMBS UP", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (102, 255, 25), 3)
                                else:
                                    finger_fold_status.append(False)
                    '''
               
                       
                    # Dislike
                    #if lm_list[thumb_tip].y > lm_list[thumb_tip - 1].y > lm_list[thumb_tip - 2].y:
                     #   cv2.putText(output_image, "THUMBS DOWN", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (102, 255, 25), 3)
                
       
        ####################################################################################################################  
        
      
        #(results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y<results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].y) and 
        #(results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y<results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP].y) and
        #(results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_TIP].y<results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.RING_FINGER_MCP].y) and
        #(results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_TIP].y<results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.PINKY_MCP].y) and
        #(results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP].x<results.multi_hand_landmarks[0].landmark[mp_hands.HandLandmark.THUMB_TIP].y)
        
           
            
         ###################################################################################################################  
        # Check if the hands gestures are specified to be written.
        if draw:
            for id, lm in enumerate(results.multi_hand_landmarks):
                lm_list.append(lm)
                finger_fold_status = []
                for tip in finger_tips:
                    x, y = int(lm_list[tip].x * w), int(lm_list[tip].y * h)
                    if lm_list[tip].x < lm_list[tip - 3].x:
                        cv2.circle(output_image, (x, y), 10, (0, 255, 0), cv2.FILLED)
                    else:
                        cv2.circle(output_image, (x, y), 10, (255, 0, 0), cv2.FILLED)
                    #if hands_gestures[hand_label]!= 'UNKNOWN' and results.multi_handedness[0].classification[0].label=='RIGHT':
                    cv2.putText(output_image, hands_gestures[hand_label] , (10, (hand_index+1) * 60),cv2.FONT_HERSHEY_PLAIN, 4, color, 5)
                    #cv2.putText(output_image, "Please Wait" , (10, (hand_index+1) * 60),cv2.FONT_HERSHEY_PLAIN, 4, color, 5)
           ################################################################################################################         
                    
    print(hands_gestures[hand_label])
    # Check if the output image is specified to be displayed.
    if display:

        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off');
    
    # Otherwise
    else:

        # Return the output image and the gestures of the both hands.
        return output_image, hands_gestures

In [6]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0)
camera_video.set(3,1280)
camera_video.set(4,960)

# Create named window for resizing purposes.
cv2.namedWindow('Hand Recognizer', cv2.WINDOW_NORMAL)

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
    
    #time_elapsed = time.time() - prev
    # Read a frame.
    ok, frame = camera_video.read()
    
    
    # Check if frame is not read properly then continue to the next iteration to read the next frame.
    if not ok:
        continue
        
    #if time_elapsed > 1./frame_rate:
        #prev = time.time()
        # Flip the frame horizontally for natural (selfie-view) visualization.
    flipped_image = cv2.flip(frame, 1)
        # Perform Hands landmarks detection on the frame.

    frame, results = detectHandsLandmarks(flipped_image, hands_videos, display=False)
    if results.multi_hand_landmarks:
        frame, fingers_statuses, count = countFingers(frame, results, draw=True, display = False)
        frame,_ = recognizeGestures(frame, fingers_statuses,results, count,draw=True, display=False)
            # Display the frame.
    cv2.imshow('Hand Recognizer',frame)

        # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF

        # Check if 'ESC' is pressed and break the loop.
    if(k == 27):
        break

# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()

[True, True, True, True]
THUMBS UP
THUMBS UP
[True, True, True, True]
THUMBS UP
THUMBS UP
[True, False, False, False]
HELLO
[True, False, False, False]
HELLO
UNITE
[True, True, True, True]
THUMBS UP
THUMBS UP
[True, True, True, True]
THUMBS UP
THUMBS UP
[True, True, True, False]
UNKNOWN
[True, False, True, False]
ROCK ON
[True, False, False, False]
ROCK ON
[True, True, True, False]
CALL ME
[False, True, True, False]
CALL ME
UNITE
[False, False, True, True]
UNITE
UNITE
[False, False, True, True]
UNITE
UNITE
[False, False, False, False]
UNITE
UNITE
[False, False, False, False]
UNITE
UNITE
[False, False, False, False]
UNITE
UNITE
[False, False, False, False]
UNITE
[False, False, False, False]
UNKNOWN
[False, False, False, False]
UNKNOWN
[False, False, False, False]
UNKNOWN
[True, False, False, False]
HELLO
[True, True, False, False]
HELLO
[True, True, False, False]
HELLO
UNITE
[False, False, True, True]
UNITE
UNITE
[True, True, True, True]
THUMBS UP
THUMBS UP
[True, False, True, True]
VIC

In [22]:
camera_video.release()
cv2.destroyAllWindows()

In [29]:
 results.multi_handedness[0].classification[0]

index: 0
score: 0.990236759185791
label: "Left"

In [30]:
 results.multi_handedness[0].classification[0].label

'Left'